In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_colesterol = pd.read_csv('./datasets/colesterol.csv')

In [ ]:
df_colesterol.info()

In [ ]:
df_colesterol.describe()

In [ ]:
df_colesterol.drop(columns=['Id'], axis=1, inplace=True)

In [ ]:
df_colesterol.head(10)

In [ ]:
df_colesterol.columns = [
  'blood_type',
  'smoking',
  'physical_activity_degree',
  'age',
  'weight',
  'height',
  'colesterol_level'
]

In [ ]:
df_colesterol.head(10)

In [ ]:
# Copiar dataframe para o EDA
df_colesterol_eda = df_colesterol.copy()

In [ ]:
# Copiar dataframe para bucketing
df_colesterol_bucketing = df_colesterol.copy()

### EDA

In [ ]:
df_colesterol_eda.blood_type.unique()

In [ ]:
df_colesterol_eda.smoking.unique()

In [ ]:
df_colesterol_eda.physical_activity_degree.unique()

In [ ]:
# Detectar valores ausentes
df_colesterol_eda.isna().sum()

In [ ]:
# Olhar para as medidas estatísticas do dataframe, 
# a fim de decidir qual parâmetro utilizar para preencher nulos
df_colesterol_eda.describe()

- age - mediana (média com valores fracionados, não adequado)
- weight - mediana (valores próximos, mas percebe-se outliers)
- height - mediana (valores muito próximos, e o número é inteiro em cm)
- Para variáveis categóricas, utilizar a moda

In [ ]:
# Coletar modas das categóricas
mode_blood_type = df_colesterol_eda.blood_type.mode()[0]
mode_smoking = df_colesterol_eda.smoking.mode()[0]
mode_physical_act = df_colesterol_eda.physical_activity_degree.mode()[0]

In [ ]:
# Coletar medianas das numéricas
median_age = df_colesterol_eda.age.median()
median_weight = df_colesterol_eda.weight.median()
median_height = df_colesterol_eda.height.median()

In [ ]:
# Input de valores ausentes
df_colesterol_eda.fillna(value={
  'blood_type': mode_blood_type,
  'smoking': mode_smoking,
  'physical_activity_degree': mode_physical_act,
  'age': median_age,
  'weight': median_weight,
  'height': median_height
}, inplace=True)

In [ ]:
df_colesterol_eda.info()

In [ ]:
# Converter idade e altura para inteiro
df_colesterol_eda.age = df_colesterol_eda.age.astype(int)
df_colesterol_eda.height = df_colesterol_eda.height.astype(int)

In [ ]:
# Verificar outliers
sns.boxplot(data=df_colesterol_eda, x='age')

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='height')

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='weight')

In [ ]:
# Filtrar o público a ser removido
df_colesterol_eda[df_colesterol_eda.weight < 40].weight.count()

In [ ]:
# Remover púbico do dataframe
df_colesterol_eda.drop(df_colesterol_eda[df_colesterol_eda.weight < 40].index, axis=0, inplace=True)

In [ ]:
df_colesterol_eda.info()

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='weight')

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='colesterol_level')

In [ ]:
# Cruzamento de variáveis categóricas com o nível de colesterol
sns.boxplot(data=df_colesterol_eda, x='blood_type', y='colesterol_level')

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='physical_activity_degree', y='colesterol_level')

In [ ]:
sns.boxplot(data=df_colesterol_eda, x='smoking', y='colesterol_level')

In [ ]:
# Cruzamento das variáveis numericas com nível de colesterol
sns.scatterplot(data=df_colesterol_eda, x='age', y='colesterol_level')

In [ ]:
sns.scatterplot(data=df_colesterol_eda, x='weight', y='colesterol_level')

In [ ]:
sns.scatterplot(data=df_colesterol_eda, x='height', y='colesterol_level')

- Verifica-se que há uma certa correlação entre valores de colesterol e valores de peso

In [ ]:
# Análise da distribuição das variáveis numéricas
sns.pairplot(data=df_colesterol_eda)

- Ajustar as variáveis categóricas para anáise de relação
  - Variáveis nominais: blood_type, smoking (não são quantificadas, apenas categorizam)
  - Variável ordinal: physical_activity_degree (baixo, moderado e alto)

In [ ]:
# Converter nominais em numéricas, usando one-hot-encoding do pandas
df_colesterol_eda = pd.get_dummies(df_colesterol_eda, columns=['blood_type', 'smoking'], dtype=int)

In [ ]:
df_colesterol_eda.head(10)

In [ ]:
# Converter variável categórica ordinal em numérica usando o factorize do Pandas
df_colesterol_eda['physical_activity_degree'] = pd.factorize(df_colesterol_eda.physical_activity_degree)[0] + 1

In [ ]:
df_colesterol_eda.head(10)

In [ ]:
# Mapa de calor com correlação
plt.figure(figsize=(15,6))
sns.heatmap(df_colesterol_eda.corr(), annot=True, vmin=-1, vmax=1)

In [ ]:
# Formato ranking, mostrar somente a correlação com o target (colesterol_level)
sns.heatmap(df_colesterol_eda.corr()[['colesterol_level']].sort_values(by='colesterol_level', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')

In [ ]:
# Bucketing idade
bins_age = [20, 30, 40, 50, 60, 70, 80]
labels_age = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-80']
df_colesterol_bucketing['age_scale'] = pd.cut(x=df_colesterol_bucketing['age'], bins=bins_age, labels=labels_age, include_lowest=True)

In [ ]:
df_colesterol_bucketing.head(10)

In [ ]:
sns.boxplot(data=df_colesterol_bucketing, x='age_scale', y='colesterol_level')

In [ ]:
# Bucketing peso
bins_weight = [40, 50, 60, 70, 80, 90, 100, 110, 120, 200]
labels_weight = ['40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120+']
df_colesterol_bucketing['weight_scale'] = pd.cut(x=df_colesterol_bucketing['weight'], bins=bins_weight, labels=labels_weight, include_lowest=True)


In [ ]:
sns.boxplot(data=df_colesterol_bucketing, x='weight_scale', y='colesterol_level')

### Treinar modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
# Criar Dataset de Treino e Teste
df_colesterol.drop(df_colesterol[df_colesterol['weight'] < 40].index, axis=0, inplace=True)

In [ ]:
df_colesterol.info()

In [ ]:
X = df_colesterol.drop(columns=['colesterol_level'])
y = df_colesterol['colesterol_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=51)

In [ ]:
X_train.shape

##### Pipeline
- Imputar moda nas variáveis categóricas - blood_type, smoking, physical_activity_degree
- Padronizar variáveis numéricas (z score) - age, height, weight
- One-hot encoding para variáveis categóricas nominais - blood_type, smoking
- Ordinal encoding para variáveis categóricas ordinais - physical_activity_degree
- Imputar mediana nas variáveis numéricas - age, height, weight

In [ ]:
nominal_columns = ['blood_type', 'smoking']
ordinal_columns = ['physical_activity_degree']
numerical_columns = ['age', 'height', 'weight']

In [ ]:
# Pipeline: transformer
nominal_transformer = Pipeline(steps=[
    ('nominal_imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
ordinal_transformer = Pipeline(steps=[
    ('ordinal_imputer', SimpleImputer(strategy='most_frequent')),
    ('encode', OrdinalEncoder(categories=[['Baixo', 'Moderado', 'Alto']], handle_unknown='error'))
])

In [ ]:
numerical_transformer = Pipeline(steps=[
    ('numerical_imputer', SimpleImputer(strategy='median')),
    ('encode', StandardScaler())
])

In [ ]:
# Criar um ColumnTransformer para encapsular todas as transformações necessárias
# Arg transformers: lista de tuplas contendo (label, transformer, colunas)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('nom', nominal_transformer, nominal_columns),
        ('ord', ordinal_transformer, ordinal_columns)
    ]
)

In [ ]:
# Criando o Pipeline principal = Pré processamento + Treinamento
regression_model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]
)

In [ ]:
# Treinar o modelo
regression_model.fit(X_train, y_train)

### Análise de métricas

In [ ]:
y_pred = regression_model.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)

### Análise de resíduos

In [ ]:
residual = y_test - y_pred

In [ ]:
from scipy.stats import zscore
residual_std = zscore(residual)

In [ ]:
sns.scatterplot(x=y_pred, y=residual_std)
plt.axhline(y=0, color='#000', linewidth=0.8)
plt.axhline(y=-2, linestyle='--', color='#c62500', linewidth=0.8)
plt.axhline(y=2, linestyle='--', color='#c62500', linewidth=0.8)

In [ ]:
# Verificar se resíduos seguem uma distribuição normal
import pingouin as pg
plt.figure(figsize=(16,8))
pg.qqplot(residual_std, dist='norm', confidence=0.95, marker='.')

In [ ]:
# Testes de normalidade
from scipy.stats import shapiro, kstest, anderson
from statsmodels.stats.diagnostic import lilliefors, het_goldfeldquandt

In [ ]:
# Teste de Shapiro-Wilk: H0 diz que os dados seguem uma distribuição normal
stat_shapiro, p_value_shapiro = shapiro(residual)
print("Estatística do teste: {} e P-Value: {}".format(stat_shapiro, p_value_shapiro))

In [ ]:
# Teste de Kolmogorov-Smirnov
stat_ks, p_value_ks = kstest(residual, 'norm')
print("Estatística do teste: {} e P-Value: {}".format(stat_ks, p_value_ks))

In [ ]:
# Teste de Lilliefors
stat_ll, p_value_ll = lilliefors(residual, dist='norm', pvalmethod='table')
print("Estatística do teste: {} e P-Value: {}".format(stat_ll, p_value_ll))

In [ ]:
# Teste de Anderson-Darling
# Se a estatística é maior que o valor crítico para uma dada confiança, rejeita-se H0. Ou seja, a distribuição não será normal
stat_ad, critical_ad, significance_ad = anderson(residual, dist='norm')

In [ ]:
critical_ad

In [ ]:
significance_ad

In [ ]:
print("Estatísitca do teste: {} e Valor crítico: {}".format(stat_ad, critical_ad[2]))

In [ ]:
# Teste de homocedasticidade de Goldfeld-Quandt
pipe = Pipeline(steps=[('preprocessor', preprocessor)])
X_test_transformed = pipe.fit_transform(X_test)

In [ ]:
X_test_transformed

In [ ]:
# H0: há homocedasticidade
goldfeld_test = het_goldfeldquandt(residual, X_test_transformed)
stat_goldfeld = goldfeld_test[0]
p_value_goldfield = goldfeld_test[1]
print("Estatística do teste: {} e P-Valor: {}".format(stat_goldfeld, p_value_goldfield))

### Realizar predições individuais

In [ ]:
individual_prediction = {
    'blood_type': 'O',
    'smoking': 'Não',
    'physical_activity_degree': 'Alto',
    'age': 40,
    'weight': 70,
    'height': 180
}

sample_df = pd.DataFrame(individual_prediction, index=[1])

In [ ]:
sample_df

In [ ]:
regression_model.predict(sample_df)

In [ ]:
# Salvar modelo
import joblib
joblib.dump(regression_model, './colesterol_model.pkl')